## Colton Lee, Eli Herigon, Jake Schildknecht

# Data Cleaning and Team Rankings

#### This is our original notebook used for data cleaning and preparation of our data. Also used to create new datasets for the plays and scores of every season.

In [1]:
import pandas as pd

In [2]:
url = "2017-2025_scores.csv"

In [3]:
df = pd.read_csv(url)

In [4]:
df.head()

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,HomeRecord,HomeScore,HomeWin,AwaySeeding,HomeSeeding,PostSeason
0,2017,Hall Of Fame,FINAL,THU,08/03,Cardinals,0-1,18.0,0.0,Cowboys,1-0,20.0,1.0,NaN,NaN,0
1,2017,Preseason Week 1,FINAL,WED,08/09,Texans,0-1,17.0,0.0,Panthers,1-0,27.0,1.0,NaN,NaN,0
2,2017,Preseason Week 1,FINAL,THU,08/10,Vikings,1-0,17.0,1.0,Bills,0-1,10.0,0.0,NaN,NaN,0
3,2017,Preseason Week 1,FINAL,THU,08/10,Falcons,0-1,20.0,0.0,Dolphins,1-0,23.0,1.0,NaN,NaN,0
4,2017,Preseason Week 1,FINAL,THU,08/10,Redskins,0-1,3.0,0.0,Ravens,1-0,23.0,1.0,NaN,NaN,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Season       3214 non-null   int64  
 1   Week         3214 non-null   object 
 2   GameStatus   3214 non-null   object 
 3   Day          2926 non-null   object 
 4   Date         2926 non-null   object 
 5   AwayTeam     3214 non-null   object 
 6   AwayRecord   3210 non-null   object 
 7   AwayScore    2926 non-null   float64
 8   AwayWin      2926 non-null   float64
 9   HomeTeam     2926 non-null   object 
 10  HomeRecord   2922 non-null   object 
 11  HomeScore    2926 non-null   float64
 12  HomeWin      2926 non-null   float64
 13  AwaySeeding  98 non-null     float64
 14  HomeSeeding  98 non-null     float64
 15  PostSeason   3214 non-null   int64  
dtypes: float64(6), int64(2), object(8)
memory usage: 401.9+ KB


In [6]:
cleandf = df.drop(columns=["Week","GameStatus","Day","Date","AwaySeeding","HomeSeeding","PostSeason"])

In [7]:
team_names = pd.unique(pd.concat([cleandf["AwayTeam"], cleandf["HomeTeam"]], ignore_index=True))
teams = pd.DataFrame(team_names, columns=["Team"])


In [8]:
teams = teams.drop_duplicates()

In [9]:
teams.tail(35)

,Team
3,Falcons
4,Redskins
5,Jaguars
6,Broncos
7,Saints
8,Eagles
9,Steelers
10,Buccaneers
11,49ers
12,Titans


In [10]:
teams = teams.drop(index=[4,32,33,34,35,37])

In [11]:
teams.head(32)

,Team
0,Cardinals
1,Texans
2,Vikings
3,Falcons
5,Jaguars
6,Broncos
7,Saints
8,Eagles
9,Steelers
10,Buccaneers


In [12]:
df_year = cleandf[cleandf['Season'] == 2024].copy()

In [13]:
#Find season wins per team
home_wins = df_year.groupby('HomeTeam')['HomeWin'].sum()
away_wins = df_year.groupby('AwayTeam')['AwayWin'].sum()
wins = home_wins.add(away_wins, fill_value=0)

#Find season losses per team
home_losses = df_year.groupby('HomeTeam')['AwayWin'].sum()
away_losses = df_year.groupby('AwayTeam')['HomeWin'].sum()

#Calculate win percent
losses = home_losses.add(away_losses, fill_value=0)
total_games = wins + losses
win_percent = round(wins / total_games, 2) * 100

#Find season total points
home_points = df_year.groupby('HomeTeam')['HomeScore'].sum()
away_points = df_year.groupby('AwayTeam')['AwayScore'].sum()
totat_points = home_points.add(away_points)

#Find season total points against
home_points_against = df_year.groupby('HomeTeam')['AwayScore'].sum()
away_points_against = df_year.groupby('AwayTeam')['HomeScore'].sum()
totat_points_against = home_points_against.add(away_points_against)

#Calculate point differential for best performing team
points_Differential = totat_points - totat_points_against

#Insert values
teams['Points'] = teams['Team'].map(totat_points).fillna(0).astype(int)
teams['PointsAgainst'] = teams['Team'].map(totat_points_against).fillna(0).astype(int)
teams['Win%'] = teams['Team'].map(win_percent).fillna(0).astype(float)
teams['PointDifferential'] = teams['Team'].map(points_Differential).fillna(0).astype(float)
teams['Wins'] = teams['Team'].map(wins).fillna(0).astype(int)
teams['Losses'] = teams['Team'].map(losses).fillna(0).astype(int)

In [14]:
sortedteams = teams.sort_values(by='PointDifferential', ascending=False)
sortedteams = sortedteams.reset_index(drop=True)
sortedteams.head(35)

,Team,Points,PointsAgainst,Win%,PointDifferential,Wins,Losses
0,Eagles,641,432,83.0,209.0,20,4
1,Lions,646,441,81.0,205.0,17,4
2,Bills,653,499,70.0,154.0,16,7
3,Broncos,531,386,62.0,145.0,13,8
4,Ravens,604,460,64.0,144.0,14,8
5,Vikings,518,397,81.0,121.0,17,4
6,Packers,525,404,62.0,121.0,13,8
7,Buccaneers,570,456,57.0,114.0,12,9
8,Commanders,619,540,65.0,79.0,15,8
9,Chargers,452,381,57.0,71.0,12,9


In [15]:
sortedteams.to_csv('2024_scores.csv', index=False)